# Auto-Encoder

Der Autoencoder versucht, eine Funktion $h_{W,b}(x) \approx x$ zu lernen. Mit anderen Worten, er versucht, eine Annäherung an die Identitätsfunktion zu lernen, um $\hat{x}$ auszugeben, die x ähnlich ist. Die Identitätsfunktion scheint eine besonders triviale Funktion zu sein, die zu lernen ist; aber indem wir dem Netzwerk Beschränkungen auferlegen, wie z.B. durch Begrenzung der Anzahl der Neuronen in Hidden Layers, können wir eine interessante Struktur über die Daten entdecken. 

Als konkretes Beispiel nehmen wir an, die Eingaben x sind die Pixel-Intensitätswerte aus einem 28×28-Bild (784 Pixel), also n=784, und es gibt $s_2$=64 Neuronen in dem Hidden Layer $L_2$. Beachten Sie, dass wir auch $y \in \mathbb{R}^{784}$ haben. 

Da es nur 128 Neuronen gibt, ist das Netzwerk gezwungen, eine "komprimierte" Darstellung der Eingabe zu lernen. D.h., wenn es nur den Vektor der Aktivierungen der Neuronen $a^{(2)} \in \mathbb{R}^{64}$ erhält, muss es versuchen, die 784-Pixel-Eingabe x zu "rekonstruieren". 

Wenn die Eingabe völlig zufällig wäre, wäre diese Komprimierungsaufgabe sehr schwierig. Wenn die Daten jedoch strukturiert sind, z.B. wenn einige der Eingabemerkmale korreliert sind, dann ist dieser Algorithmus in der Lage, einige dieser Korrelationen zu entdecken.

## Fashion MNIST Dataset

`Fashion-MNIST` ist ein Datensatz von Zalandos Artikelbildern - bestehend aus einem Trainingssatz mit 60.000 Beispielen und einem Testsatz mit 10.000 Beispielen. 

Jedes Beispiel ist ein Graustufenbild im Format 28x28, das mit einem Label aus 10 Klassen verbunden ist. Fashion-MNIST ist ein direkter Drop-in-Ersatz für den ursprünglichen MNIST-Datensatz von handgeschriebenen Ziffern. Der Datensatz hat die gleiche Bildgröße und die gleiche Struktur von Trainings- und Test-Splits.

Jedes Bild trägt einen der folgenden Label:

Label | Description
--- | ---
0|T-shirt/top
1|Trouser
2|Pullover
3|Dress
4|Coat
5|Sandal
6|Shirt
7|Sneaker
8|Bag
9|Ankle boot


In [ ]:
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf_config.allow_soft_placement = True

from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# MNIST Dataset parameters.
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.01
training_steps = 5000
batch_size = 64
display_step = training_steps / 10

# Network Parameters
num_hidden_1 = 128 # 1st layer num features.
num_hidden_2 = 64 # 2nd layer num features (the latent dim).

### Daten laden

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

### Daten vorbereiten

Jedes Bild wird konvertiert zu float32, normalisiert auf das Intervall [0, 1] and ausgestreckt auf ein 1-dimensionales Array von 784 Features (28*28).

In [ ]:
# Convert to float32.
x_train, x_test = x_train.astype(np.float32), x_test.astype(np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

Wir erzeugen uns wieder einen iterierbaren **`tf.data.Dataset`** wie wir es schon beim Neuralen Netz aus den MNIST Ziffern gesehen hatten:

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(batch_size).prefetch(1)

test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.repeat().batch(batch_size).prefetch(1)


## Autoencoder

Als nächstes definieren wir unser Autoencoder Netzwerk, diesmal allerdings nicht mehr der High-Level API Keras, sondern mit den Tensorflow zu Fuß.

Wir verwenden jeweils zwei Hidden Layer für das En- und Decoding, gehen dabei über 128 Neuronen auf 64 Neuronen.

![](autoencode.png)

### Variablen für Weight & Bias der Layer

`tf.Variable` sind die Elemente, die der Algorithmus trainieren wird, also unsere Weights und Bias:


In [ ]:
# A random value generator to initialize weights.
random_normal = tf.initializers.RandomNormal()

weights = {
    'encoder_h1': tf.Variable(random_normal([num_features, num_hidden_1])),
    'encoder_h2': tf.Variable(random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(random_normal([num_hidden_1, num_features])),
}
biases = {
    'encoder_b1': tf.Variable(random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(random_normal([num_features])),
}

### Building the encoder.

In [ ]:
def encoder(x):
    # Encoder Hidden layer with sigmoid activation.
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    
    # Encoder Hidden layer with sigmoid activation.
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    
    return layer_2

### Building the decoder

In [ ]:
def decoder(x):
    # Decoder Hidden layer with sigmoid activation.
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    
    # Decoder Hidden layer with sigmoid activation.
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    
    return layer_2

### Mean square loss between original images and reconstructed ones

In [ ]:
def mean_square(reconstructed, original):
    return tf.reduce_mean(tf.pow(original - reconstructed, 2))

### Adam optimizer

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

### Optimization process

TensorFlow bietet die `tf.GradientTape` API für die **automatische Differenzierung**, d.h. die Berechnung des Gradienten einer Berechnung in Bezug auf seine Eingangsvariablen. 

TensorFlow zeichnet alle Operationen, die im Kontext eines tf.GradientTape ausgeführt werden, auf einem "Band" auf. 

TensorFlow verwendet dann dieses Band und die Gradienten, die jeder aufgezeichneten Operation zugeordnet sind, um die Gradienten einer "aufgezeichneten" Berechnung zu berechnen.

In [ ]:
def run_optimization(x):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        reconstructed_image = decoder(encoder(x))
        loss = mean_square(reconstructed_image, x)

    # Variables to update, i.e. trainable variables.
    trainable_variables = {**weights, **biases}.values()
    
    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    
    return loss

### Trainiere das Modell

`take(count)` holt sich die nächsten `count` Batch-Elemente aus dem Dataset-Strom.


In [ ]:
train_loss_results = []

for step, (batch_x, _) in enumerate(train_data.take(training_steps + 1)):
    
    # Run the optimization.
    loss = run_optimization(batch_x)
    train_loss_results.append(loss)
    
    if step % display_step == 0:
        print("step: %i, loss: %f" % (step, loss))

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 6))

ax.set_ylabel("Loss", fontsize=14)
ax.set_xlabel("Epoch", fontsize=14)
ax.plot(train_loss_results);

## Testing and Visualization

In [ ]:
import matplotlib.pyplot as plt

# Encode and decode images from test set and visualize their reconstruction.
n = 16

canvas_orig = np.empty((28, 28 * n))
canvas_encod = np.ones((28, 28 * n))
canvas_recon = np.empty((28, 28 * n))

for i, (batch_x, _) in enumerate(test_data.take(n)):
    # Encode and decode the digit image.
    encoded_images = encoder(batch_x)
    reconstructed_images = decoder(encoded_images)

    part = slice(i * 28, (i + 1) * 28)
    part_encod = slice(i * 28+10, i * 28 + 18)

    # Display original images.
    img = batch_x[i].numpy().reshape([28, 28])
    canvas_orig[0:28, part] = img

    # Display Encodings.
    img = encoded_images[i].numpy().reshape([8, 8])
    canvas_encod[10:18, part_encod] = img

    # Display reconstructed images.
    reconstr_img = reconstructed_images[i].numpy().reshape([28, 28])
    canvas_recon[0:28, part] = reconstr_img

print("Original Images")
plt.figure(figsize=(n, n))
plt.axis('off')
plt.imshow(canvas_orig, origin="upper", cmap="gray")
plt.show()

print("Encoded Images")     
plt.figure(figsize=(n, n))
plt.axis('off')
plt.imshow(canvas_encod, origin="upper", cmap="gray")
plt.show()

print("Reconstructed Images")
plt.figure(figsize=(n, n))
plt.axis('off')
plt.imshow(canvas_recon, origin="upper", cmap="gray")
plt.show()